# Monitor the training process -- Weather Prediction

## 0. Import packages and modules

In [ ]:
!pwd

In [ ]:
!hostname

In [ ]:
import os
# Limit log messages from Tensorflow
# 0 = all messages are logged (default behavior)
# 1 = INFO messages are not printed
# 2 = INFO and WARNING messages are not printed
# 3 = INFO, WARNING, and ERROR messages are not printed
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '1' 

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

import sklearn
import tensorflow
import keras

print(sklearn.__version__)
print(tensorflow.__version__)
print(keras.__version__)

In [ ]:
# print GPU info

import tensorflow as tf

print(tf.config.list_physical_devices('GPU'))

print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

## 1. Formulate / Outline the problem: weather prediction

In [ ]:
data = pd.read_csv('./weather_prediction_dataset.csv')
data.head()

## 2. Identify inputs and outputs

In [ ]:
data

In [ ]:
data.columns

In [ ]:
data.shape

## 3. Prepare data

In [ ]:
nr_rows = 365*3 # 3 years

# data
X_data = data.loc[:nr_rows] # Select first 3 years
X_data = X_data.drop(columns=['DATE', 'MONTH']) # Drop date and month column

# labels (sunshine hours the next day)
y_data = data.loc[1:(nr_rows + 1)]["BASEL_sunshine"]

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_holdout, y_train, y_holdout = train_test_split(X_data, y_data, test_size=0.3, random_state=0)
X_val, X_test, y_val, y_test = train_test_split(X_holdout, y_holdout, test_size=0.5, random_state=0)

## 4. Building architecture from scratch

In [ ]:
def create_nn(input_shape):
    # Input layer
    inputs = keras.Input(shape=input_shape, name='input')

    # Dense layers
    layers_dense = keras.layers.Dense(100, 'relu')(inputs)
    layers_dense = keras.layers.Dense(50, 'relu')(layers_dense)

    # Output layer
    outputs = keras.layers.Dense(1)(layers_dense)

    return keras.Model(inputs=inputs, outputs=outputs, name="weather_prediction_model")

model = create_nn(input_shape=(X_data.shape[1],))

In [ ]:
model.summary()

## 5. Choose a loss function and optimizer

In [ ]:
# loss function

model.compile(loss='mse')

In [ ]:
# optimizer

model.compile(optimizer='adam', loss='mse')

In [ ]:
model.compile(optimizer='adam', loss='mse', metrics=[keras.metrics.RootMeanSquaredError()])

In [ ]:
def compile_model(model):
    model.compile(optimizer='adam', loss='mse',
                  metrics=[keras.metrics.RootMeanSquaredError()])

compile_model(model)

## 6. Train the model

In [ ]:
history = model.fit(X_train, y_train,
                    batch_size=32,
                    epochs=200,
                    verbose=2)

In [ ]:
def plot_history(history, metrics):
    """
    Plot the training history

    Args:
        history (keras History object that is returned by model.fit())
        metrics (str, list): Metric or a list of metrics to plot
    """
    history_df = pd.DataFrame.from_dict(history.history)
    sns.lineplot(data=history_df[metrics])
    plt.xlabel("epochs")
    plt.ylabel("metric")

plot_history(history, 'root_mean_squared_error')

## 7. Perform a Prediction/Classification

In [ ]:
y_train_predicted = model.predict(X_train)
y_test_predicted = model.predict(X_test)

## 8. Measure performance

In [ ]:
def plot_predictions(y_pred, y_true, title):
    plt.style.use('ggplot')  # optional, that's only to define a visual style
    plt.scatter(y_pred, y_true, s=10, alpha=0.5)
    plt.axline((0,0),slope = 1, color = "black") # plot diagonal reference line
    plt.xlabel("predicted sunshine hours")
    plt.ylabel("true sunshine hours")
    plt.title(title)

plot_predictions(y_train_predicted, y_train, title='Predictions on the training set')

In [ ]:
plot_predictions(y_test_predicted, y_test, title='Predictions on the test set')

In [ ]:
train_metrics = model.evaluate(X_train, y_train, return_dict=True)
test_metrics = model.evaluate(X_test, y_test, return_dict=True)

print('Train RMSE: {:.2f}, Test RMSE: {:.2f}'.format(train_metrics['root_mean_squared_error'], test_metrics['root_mean_squared_error']))

In [ ]:
y_baseline_prediction = X_test['BASEL_sunshine']
plot_predictions(y_baseline_prediction, y_test, title='Baseline predictions on the test set')

In [ ]:
from sklearn.metrics import root_mean_squared_error

rmse_baseline = root_mean_squared_error(y_test, y_baseline_prediction)
print('Baseline:', rmse_baseline)
print('Neural network: ', test_metrics['root_mean_squared_error'])

## 9. Refine the model

In [ ]:
model = create_nn(input_shape=(X_data.shape[1],))
compile_model(model)

In [ ]:
history = model.fit(X_train, y_train,
                    batch_size=32,
                    epochs=200,
                    validation_data=(X_val, y_val))

In [ ]:
plot_history(history, ['root_mean_squared_error', 'val_root_mean_squared_error'])

### Early stopping: stop when things are looking best

In [ ]:
model = create_nn(input_shape=(X_data.shape[1],))
compile_model(model)

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping

earlystopper = EarlyStopping(
    monitor='val_loss',
    patience=10
    )

history = model.fit(X_train, y_train,
                    batch_size = 32,
                    epochs = 200,
                    validation_data=(X_val, y_val),
                    callbacks=[earlystopper])

In [ ]:
plot_history(history, ['root_mean_squared_error', 'val_root_mean_squared_error'])

### BatchNorm: the “standard scaler” for deep learning

In [ ]:
def create_nn(input_shape):
    # Input layer
    inputs = keras.layers.Input(shape=input_shape, name='input')

    # Dense layers
    layers_dense = keras.layers.BatchNormalization()(inputs) # This is new!
    layers_dense = keras.layers.Dense(100, 'relu')(layers_dense)
    layers_dense = keras.layers.Dense(50, 'relu')(layers_dense)

    # Output layer
    outputs = keras.layers.Dense(1)(layers_dense)

    # Defining the model and compiling it
    return keras.Model(inputs=inputs, outputs=outputs, name="model_batchnorm")


model = create_nn(input_shape=(X_data.shape[1],))
compile_model(model)
model.summary()

In [ ]:
history = model.fit(X_train, y_train,
                    batch_size = 32,
                    epochs = 1000,
                    validation_data=(X_val, y_val),
                    callbacks=[earlystopper])

plot_history(history, ['root_mean_squared_error', 'val_root_mean_squared_error'])

### Run on test set and compare to naive baseline

In [ ]:
y_test_predicted = model.predict(X_test)
plot_predictions(y_test_predicted, y_test, title='Predictions on the test set')

## 10. Save model

In [ ]:
model.save('weather_prediction.keras')